In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import numpy as np
from utils.helpers import get_standard_stats  
fs = 14

# microbio

In [ ]:
# two seperate models next: one trained and tested on all episodes (False) and one trained and tested only on episodes with microbiology (True)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme()
                     

metrics = ['Precision', 'Recall', 'F1', 'Balanced Accuracy', 'AUPRC', 'AUROC']
results_df = pd.DataFrame()

                   


microbio = [True, False]
for seed in [42, 43, 44, 45, 46]:  
    episodes = pd.read_parquet("data/episodes/mimic/microbiology_res_False/seed_"+str(seed)+"/test_data.parquet")
    cols = ['starttime']
    episodes[cols] = episodes[cols].apply(lambda x: x.dt.floor('5min')) 
    for m in microbio:
        print("data/results/lstm/mimic/microbiology_res_"+str(m)+"/ab_False/use_censored_True/lookback_7/aggregated_hours_4/seed_"+str(seed)+"/dropout_0-0/lambda_0-1/num_lin_layers_2/num_stacked_lstm_3/hidden_dim_256/lr_0-01/bs_128/is_tuned_False/use_relus_False/use_bn_False/test_res.csv")
        #df = pd.read_csv("data/results/lstm/mimic/microbiology_res_"+str(m)+"/ab_False/use_censored_True/lookback_7/aggregated_hours_4/seed_"+str(seed)+"/dropout_0-0/lambda_0-1/num_lin_layers_2/num_stacked_lstm_3/hidden_dim_256/lr_0-01/bs_128/is_tuned_False/use_relus_False/use_bn_False/test_res.csv")
        df = pd.read_csv("data/results/lstm/mimic/microbiology_res_"+str(m)+"/ab_False/use_censored_True/lookback_7/aggregated_hours_4/seed_"+str(seed)+"/dropout_0-0/lambda_0-1/num_lin_layers_2/num_stacked_lstm_3/hidden_dim_256/lr_0-01/bs_128/is_tuned_False/use_relus_False/use_bn_False/test_gt_and_preds.csv")
        df['starttime'] = pd.to_datetime(df['starttime'])
        df[['starttime']] = df[['starttime']].apply(lambda x: x.dt.floor('5min')) 
        df = pd.merge(df, episodes[['starttime','ID','isPositive']], on=['starttime', 'ID'], how='inner')
        #df = df[df['isPositive'].notna()]
        df = get_standard_stats(gt=df['next_day'], preds=df['pred'], preds_proba=df['True'])
        df.rename({'balanced_accuracy': 'Balanced Accuracy', 'prc_auc':'AUPRC', 'roc_auc':'AUROC', 'f1':'F1', 'recall':'Recall', 'precision':'Precision'}, inplace=True, axis=1)
        df['Microbiology restriction'] = m
        
        results_df = pd.concat([results_df, df])
        

mean_std_df = results_df.groupby('Microbiology restriction').agg({metric: ['mean', 'std'] for metric in metrics}).reset_index()



display(mean_std_df)

for_latex = mean_std_df.set_index(('Microbiology restriction', ''))
for_latex.index.rename('Microbiology restriction', inplace=True)
display(for_latex)
print(for_latex.transpose().to_latex(float_format="%.2f", bold_rows=True, caption='Next Day Model trained & tested on all vs. only trained and tested on episodes with microbiology result'))
print(for_latex[['Balanced Accuracy', 'AUPRC', 'AUROC']].to_latex(float_format="%.2f", bold_rows=True, caption='Model trained & tested on all vs. only on episodes with microbiology result'))
print(for_latex[['Precision', 'Recall', 'F1']].to_latex(float_format="%.2f", bold_rows=True, caption='Model trained & tested on all vs. only on episodes with microbiology result'))

# Visualisierung der Metriken einzeln
def plot_metric(metric):
    plt.figure(figsize=(5, 5))
    sns.barplot(x='Microbiology restriction', y=(metric, 'mean'), data=mean_std_df, capsize=.1, color='skyblue')
    plt.errorbar(x=np.arange(len(microbio)), y=mean_std_df[(metric, 'mean')], yerr=mean_std_df[(metric, 'std')], fmt='none', c='black', capsize=5)
    #plt.title(f'Mean and SD of {metric.capitalize()}')
    plt.xlabel('Microbiology restriction', fontsize=fs)
    plt.ylabel(metric, fontsize=fs)
    plt.ylim([0, 1])
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.tight_layout()
    #plt.savefig('images/experiments/microbiology/'+'microbiology_tested_all_'+metric+".png")
    plt.show()

# Plotte jede Metrik einzeln
for metric in metrics:
    plot_metric(metric)

In [ ]:
# two seperate models: one trained on all episodes and tested only on microbiology restricted, the other trained only on microbiology restricted and tested only on microbiology restricted

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from utils.helpers import get_standard_stats  

sns.set_theme()

              

metrics = ['Precision', 'Recall', 'F1', 'Balanced Accuracy', 'AUPRC', 'AUROC']
results_df = pd.DataFrame()

microbio = [True, False]
for seed in [42, 43, 44, 45, 46]:  
    episodes = pd.read_parquet("data/episodes/mimic/microbiology_res_False/seed_"+str(seed)+"/test_data.parquet")
    cols = ['starttime']
    episodes[cols] = episodes[cols].apply(lambda x: x.dt.floor('5min'))      

    for m in microbio:
        df = pd.read_csv("data/results/lstm/mimic/microbiology_res_"+str(m)+"/ab_False/use_censored_True/lookback_7/aggregated_hours_4/seed_"+str(seed)+"/dropout_0-0/lambda_0-1/num_lin_layers_2/num_stacked_lstm_3/hidden_dim_256/lr_0-01/bs_128/is_tuned_False/use_relus_False/use_bn_False/test_gt_and_preds.csv")
        df['starttime'] = pd.to_datetime(df['starttime'])
        df[['starttime']] = df[['starttime']].apply(lambda x: x.dt.floor('5min')) 
        df = pd.merge(df, episodes[['starttime','ID','isPositive']], on=['starttime', 'ID'], how='inner')
        df = df[df['isPositive'].notna()]
        df = get_standard_stats(gt=df['next_day'], preds=df['pred'], preds_proba=df['True'])
        df.rename({'balanced_accuracy': 'Balanced Accuracy', 'prc_auc':'AUPRC', 'roc_auc':'AUROC', 'f1':'F1', 'recall':'Recall', 'precision':'Precision'}, inplace=True, axis=1)
        df['Microbiology restriction'] = m
        
        results_df = pd.concat([results_df, df])
        

mean_std_df = results_df.groupby('Microbiology restriction').agg({metric: ['mean', 'std'] for metric in metrics}).reset_index()



display(mean_std_df)

for_latex = mean_std_df.set_index(('Microbiology restriction', ''))
for_latex.index.rename('Microbiology restriction', inplace=True)
display(for_latex)
print(for_latex.transpose().to_latex(float_format="%.2f", bold_rows=True, caption='Next Day model trained on all & tested only on episodes with microbiology restriction vs. trainend and tested only on episodes with microbiology restriction'))
#print(for_latex[['Balanced Accuracy', 'AUPRC', 'AUROC']].to_latex(float_format="%.2f", bold_rows=True, caption='Model trained & tested on all vs. only uncensored'))


# Visualisierung der Metriken einzeln
def plot_metric(metric):
    plt.figure(figsize=(5, 5))
    sns.barplot(x='Microbiology restriction', y=(metric, 'mean'), data=mean_std_df, capsize=.1, color='skyblue')
    plt.errorbar(x=np.arange(len(microbio)), y=mean_std_df[(metric, 'mean')], yerr=mean_std_df[(metric, 'std')], fmt='none', c='black', capsize=5)
    #plt.title(f'Mean and SD of {metric.capitalize()}')
    plt.xlabel('Microbiology restriction', fontsize=fs)
    plt.ylabel(metric, fontsize=fs)
    plt.ylim([0, 1])
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.tight_layout()
    #plt.savefig('images/experiments/censored/'+'censored_tested_all_'+metric+".png")
    plt.show()

# Plotte jede Metrik einzeln
for metric in metrics:
    plot_metric(metric)

In [ ]:
# two seperate models traditional
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme()
                     

metrics = ['Precision', 'Recall', 'F1', 'Balanced Accuracy', 'AUPRC', 'AUROC']
results_df = pd.DataFrame()

microbio = [True, False]
for seed in [42, 43, 44, 45, 46]:  
    episodes = pd.read_parquet("data/episodes/mimic/microbiology_res_False/seed_"+str(seed)+"/test_data.parquet")
    cols = ['starttime']
    episodes[cols] = episodes[cols].apply(lambda x: x.dt.floor('5min'))   
    for m in microbio:
        #df = pd.read_csv("data/results/traditional/mimic/microbiology_res_"+str(m)+"/ab_False/seed_"+str(seed)+"/lookback_2/time_point('random', 1)/sample_None_None/LGBMClassifier/test_res.csv")
        #print("data/results/traditional/mimic/microbiology_res_"+str(m)+"/ab_False/seed_"+str(seed)+"/lookback_2/time_point('random', 1)/sample_None_None/LGBMClassifier/test_res.csv")
        df = pd.read_csv("data/results/traditional/mimic/microbiology_res_"+str(m)+"/ab_False/seed_"+str(seed)+"/lookback_2/time_point('random', 1)/sample_None_None/LGBMClassifier/test_gt_and_preds.csv")
        df['starttime'] = pd.to_datetime(df['starttime'])
        df[['starttime']] = df[['starttime']].apply(lambda x: x.dt.floor('5min')) 
        df = pd.merge(df, episodes[['starttime','ID','isPositive']], on=['starttime', 'ID'], how='inner')
        #if m == True:
        #    df = df[df['isPositive'].notna()]
        df = get_standard_stats(gt=df['lot<5d'], preds=df['pred'], preds_proba=df['True'])
        df.rename({'balanced_accuracy': 'Balanced Accuracy', 'prc_auc':'AUPRC', 'roc_auc':'AUROC', 'f1':'F1', 'recall':'Recall', 'precision':'Precision'}, inplace=True, axis=1)
        df['Microbiology restriction'] = m
        
        results_df = pd.concat([results_df, df])
        

mean_std_df = results_df.groupby('Microbiology restriction').agg({metric: ['mean', 'std'] for metric in metrics}).reset_index()



display(mean_std_df)

for_latex = mean_std_df.set_index(('Microbiology restriction', ''))
for_latex.index.rename('Microbiology restriction', inplace=True)
display(for_latex)
print(for_latex[['Balanced Accuracy', 'AUPRC', 'AUROC']].to_latex(float_format="%.2f", bold_rows=True, caption='Traditional Model trained & tested on all vs. only trained and tested on episodes with microbiology result'))
print(for_latex[['Precision', 'Recall', 'F1']].to_latex(float_format="%.2f", bold_rows=True, caption='Traditional Model trained & tested on all vs. only trained and tested on episodes with microbiology result'))


def plot_metric(metric):
    plt.figure(figsize=(5, 5))
    sns.barplot(x='Microbiology restriction', y=(metric, 'mean'), data=mean_std_df, capsize=.1, color='skyblue')
    plt.errorbar(x=np.arange(len(microbio)), y=mean_std_df[(metric, 'mean')], yerr=mean_std_df[(metric, 'std')], fmt='none', c='black', capsize=5)
    #plt.title(f'Mean and SD of {metric.capitalize()}')
    plt.xlabel('Microbiology restriction', fontsize=fs)
    plt.ylabel(metric, fontsize=fs)
    plt.ylim([0, 1])
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.tight_layout()
    plt.savefig('images/experiments/microbiology/'+'trad_microbiology_all_'+metric+".png")
    plt.show()

for metric in metrics:
    plot_metric(metric)

In [ ]:
# one model traditional
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


sns.set_theme()

              

metrics = ['Precision', 'Recall', 'F1', 'Balanced Accuracy', 'AUPRC', 'AUROC']
results_df = pd.DataFrame()

microbio = [True, False]
for seed in [42, 43, 44, 45, 46]:  #
    episodes = pd.read_parquet("data/episodes/mimic/microbiology_res_False/seed_"+str(seed)+"/test_data.parquet")
    cols = ['starttime']
    episodes[cols] = episodes[cols].apply(lambda x: x.dt.floor('5min'))      
    for m in microbio:
        df = pd.read_csv("data/results/traditional/mimic/microbiology_res_"+str(m)+"/ab_False/seed_"+str(seed)+"/lookback_2/time_point('random', 1)/sample_None_None/LGBMClassifier/test_gt_and_preds.csv")
        df['starttime'] = pd.to_datetime(df['starttime'])
        df[['starttime']] = df[['starttime']].apply(lambda x: x.dt.floor('5min')) 
        df = pd.merge(df, episodes[['starttime','ID','isPositive']], on=['starttime', 'ID'], how='inner')
        df = df[df['isPositive'].notna()]
        df = get_standard_stats(gt=df['lot<5d'], preds=df['pred'], preds_proba=df['True'])
        df.rename({'balanced_accuracy': 'Balanced Accuracy', 'prc_auc':'AUPRC', 'roc_auc':'AUROC', 'f1':'F1', 'recall':'Recall', 'precision':'Precision'}, inplace=True, axis=1)
        df['Microbiology restriction'] = m
        
        results_df = pd.concat([results_df, df])
        

mean_std_df = results_df.groupby('Microbiology restriction').agg({metric: ['mean', 'std'] for metric in metrics}).reset_index()



display(mean_std_df)

for_latex = mean_std_df.set_index(('Microbiology restriction', ''))
for_latex.index.rename('Microbiology restriction', inplace=True)
display(for_latex)
print(for_latex[['Balanced Accuracy', 'AUPRC', 'AUROC']].to_latex(float_format="%.2f", bold_rows=True, caption='Traditional model trained on all & tested only on episodes with microbiology restriction vs. trainend and tested only on episodes with microbiology restriction'))
print(for_latex[['Precision', 'Recall', 'F1']].to_latex(float_format="%.2f", bold_rows=True, caption='Traditional model trained on all & tested only on episodes with microbiology restriction vs. trainend and tested only on episodes with microbiology restriction'))


def plot_metric(metric):
    plt.figure(figsize=(5, 5))
    sns.barplot(x='Microbiology restriction', y=(metric, 'mean'), data=mean_std_df, capsize=.1, color='skyblue')
    plt.errorbar(x=np.arange(len(microbio)), y=mean_std_df[(metric, 'mean')], yerr=mean_std_df[(metric, 'std')], fmt='none', c='black', capsize=5)
    #plt.title(f'Mean and SD of {metric.capitalize()}')
    plt.xlabel('Microbiology restriction', fontsize=fs)
    plt.ylabel(metric, fontsize=fs)
    plt.ylim([0, 1])
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.tight_layout()
    plt.savefig('images/experiments/microbiology/'+'trad_microbiology_restricted_'+metric+".png")
    plt.show()


for metric in metrics:
    plot_metric(metric)

In [ ]:

episodes = pd.read_parquet("data/episodes/mimic/microbiology_res_False/seed_"+str(seed)+"/test_data.parquet")

y = pd.read_parquet("data/model_input/traditional/mimic/microbiology_res_False/ab_False/seed_"+str(seed)+"/y_test_time_point_('random', 1)_lookback_2.parquet")
display(y)
cols = ['AdmissionDateTime_hosp','endtime','starttime','lot','DischargeDateTime_hosp','AdmissionDateTime_icu','DischargeDateTime_icu']
episodes[cols] = episodes[cols].apply(lambda x: x.dt.floor('5min')) 

y = pd.merge(y, episodes[['starttime','ID','isPositive']], on=['starttime', 'ID'], how='inner')
display(y)

## antibiotics

In [ ]:
base_path_template = "data/results/traditional/mimic/microbiology_res_False/ab_{}/seed_{}/lookback_2/time_point('random', 1)/sample_None_None/LGBMClassifier/test_res.csv"
metrics = ['Precision', 'Recall', 'F1', 'Balanced Accuracy', 'AUPRC', 'AUROC']
results_df = pd.DataFrame()
for seed in range(42, 47):  
    for ab in [True, False]: 
        path = base_path_template.format(ab, seed)
        df = pd.read_csv(path)
        df['Include Antibiotics'] = ab
        df.rename({'balanced_accuracy': 'Balanced Accuracy', 'prc_auc':'AUPRC', 'roc_auc':'AUROC', 'f1':'F1', 'recall':'Recall', 'precision':'Precision'}, inplace=True, axis=1)
        results_df = pd.concat([results_df, df])

mean_std_df = results_df.groupby('Include Antibiotics').agg({metric: ['mean', 'std'] for metric in metrics}).reset_index()

for_latex = mean_std_df.set_index(('Include Antibiotics', ''))
for_latex.index.rename('Include Antibiotics', inplace=True)
display(for_latex)
print(for_latex[['Balanced Accuracy', 'AUPRC', 'AUROC']].to_latex(float_format="%.2f", bold_rows=True, caption='Performance when antibiotics are included as features'))
print(for_latex[['Precision', 'Recall', 'F1']].to_latex(float_format="%.2f", bold_rows=True, caption='Performance when antibiotics are included as features'))



def plot_metric(metric):
    plt.figure(figsize=(5, 5))
    sns.barplot(x='Include Antibiotics', y=(metric, 'mean'), data=mean_std_df, capsize=.1, color='skyblue')
    plt.errorbar(x=np.arange(2), y=mean_std_df[(metric, 'mean')], yerr=mean_std_df[(metric, 'std')], fmt='none', c='black', capsize=5)
    #plt.title(f'Mean and SD of {metric.capitalize()}')
    plt.xlabel('Include Antibiotics', fontsize=fs)
    plt.ylabel(metric, fontsize=fs)
    plt.ylim([0, 1])
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.tight_layout()
    plt.savefig('images/experiments/antibiotics/'+'trad_antibiotics_'+metric+".png")
    plt.show()


for metric in metrics:
    plot_metric(metric)


In [ ]:
#base_path_template = "data/results/traditional/mimic/microbiology_res_False/ab_{}/seed_{}/lookback_2/time_point('random', 1)/sample_None_None/LGBMClassifier/test_res.csv"
base_path_template = "data/results/lstm/mimic/microbiology_res_False/ab_{}/use_censored_True/lookback_7/aggregated_hours_4/seed_{}/dropout_0-0/lambda_0-1/num_lin_layers_2/num_stacked_lstm_3/hidden_dim_256/lr_0-01/bs_128/is_tuned_False/use_relus_False/use_bn_False/test_res.csv"
metrics = ['Precision', 'Recall', 'F1', 'Balanced Accuracy', 'AUPRC', 'AUROC']
results_df = pd.DataFrame()
for seed in range(42, 47):  
    for ab in [True, False]: 
        path = base_path_template.format(ab, seed)
        df = pd.read_csv(path)
        df['Include Antibiotics'] = ab
        df.rename({'balanced_accuracy': 'Balanced Accuracy', 'prc_auc':'AUPRC', 'roc_auc':'AUROC', 'f1':'F1', 'recall':'Recall', 'precision':'Precision'}, inplace=True, axis=1)
        results_df = pd.concat([results_df, df])

mean_std_df = results_df.groupby('Include Antibiotics').agg({metric: ['mean', 'std'] for metric in metrics}).reset_index()

for_latex = mean_std_df.set_index(('Include Antibiotics', ''))
for_latex.index.rename('Include Antibiotics', inplace=True)
display(for_latex)
print(for_latex[['Balanced Accuracy', 'AUPRC', 'AUROC']].to_latex(float_format="%.2f", bold_rows=True, caption='Performance when antibiotics are included as features'))
print(for_latex[['Precision', 'Recall', 'F1']].to_latex(float_format="%.2f", bold_rows=True, caption='Performance when antibiotics are included as features'))



def plot_metric(metric):
    plt.figure(figsize=(5, 5))
    sns.barplot(x='Include Antibiotics', y=(metric, 'mean'), data=mean_std_df, capsize=.1, color='skyblue')
    plt.errorbar(x=np.arange(2), y=mean_std_df[(metric, 'mean')], yerr=mean_std_df[(metric, 'std')], fmt='none', c='black', capsize=5)
    #plt.title(f'Mean and SD of {metric.capitalize()}')
    plt.xlabel('Include Antibiotics', fontsize=fs)
    plt.ylabel(metric, fontsize=fs)
    plt.ylim([0, 1])
    plt.xticks(fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.tight_layout()
    plt.savefig('images/experiments/antibiotics/'+'nd_antibiotics_'+metric+".png")
    plt.show()


for metric in metrics:
    plot_metric(metric)
